In [1]:
import os
import pandas as pd
import numpy as np
import ee

In [2]:
# 认证并初始化Google Earth Engine
ee.Authenticate()  # 只需要在第一次运行时执行
ee.Initialize(project= 'ee-zongrong-flood')

In [11]:
# Load the image for 2020 and select the built_surface band
image_2020 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/2020').select('built_surface')

# Calculate the min and max built_surface values for the 2020 image
min_max = image_2020.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=ee.Geometry.BBox(-180, -90, 180, 90),
    scale=30,
    bestEffort=True
).getInfo()

min_value = min_max['built_surface_min']
max_value = min_max['built_surface_max']

print(f"2020 - Min value: {min_value}, Max value: {max_value}")

2020 - Min value: 0, Max value: 9509


In [12]:
# Function to get built-up surface value at given coordinates and normalize it using logarithmic transformation
def get_built_value(coords):
    point = ee.Geometry.Point(coords)
    built_value = image_2020.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
    ).get('built_surface')
    
    value = built_value.getInfo()
    if value is None:
        # If the value is None, use nearest neighbor interpolation to fill the gap
        built_value = image_2020.reduceNeighborhood(
            reducer=ee.Reducer.mean(),
            kernel=ee.Kernel.square(5),  # Increase the kernel size
            inputWeight=True,
            optimization='boxcar'
        ).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=30,
            bestEffort=True
        ).get('built_surface')
        
        value = built_value.getInfo()
        if value is None:
            return 0
    
    # Apply logarithmic transformation before normalization
    log_value = np.log(value + 1)  # Adding 1 to avoid log(0)
    log_min_value = np.log(min_value + 1)
    log_max_value = np.log(max_value + 1)
    
    normalized_value = (log_value - log_min_value) / (log_max_value - log_min_value) * 10
    print(f"Coordinates: {coords}, Original value: {value}, Normalized value: {normalized_value}")  # Debugging line
    return normalized_value

# Specify the input and output directories
input_dir = 'C:/Users/lzr/SSCI/StreetviewLLM/groudtruth/Sampling'
output_dir = 'C:/Users/lzr/SSCI/StreetviewLLM/groudtruth/impervious_surface'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each CSV file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        input_csv = os.path.join(input_dir, filename)
        df = pd.read_csv(input_csv)
        
        # Create a new column for built-up surface values for 2020
        df['Built_Surface_2020'] = df.apply(lambda row: get_built_value([row['longitude'], row['latitude']]), axis=1)
        
        # Define the output filename
        output_csv = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_impervious_surface.csv")
        
        # Save the new dataframe to a CSV file
        df.to_csv(output_csv, index=False)
        
        print(f"Processed {input_csv} and saved with built-up surface values as {output_csv}")

print("All CSV files processed.")

Coordinates: [114.276307, 22.2695341], Original value: 0, Normalized value: 0.0
Coordinates: [114.1023043902492, 22.37115357347453], Original value: 2120, Normalized value: 8.361965110315017
Coordinates: [114.23624955805002, 22.287083445560008], Original value: 0, Normalized value: 0.0
Coordinates: [114.2983256, 22.2618995], Original value: 0, Normalized value: 0.0
Coordinates: [114.18624955824998, 22.34541677866001], Original value: 1835, Normalized value: 8.204435829317024
Coordinates: [114.1940713503955, 22.29561213573655], Original value: 0, Normalized value: 0.0
Coordinates: [114.1706041480453, 22.26166046887501], Original value: 918, Normalized value: 7.448921683592698
Coordinates: [114.25291622465, 22.30375011216001], Original value: 1901, Normalized value: 8.242990730661669
Coordinates: [114.21958289145, 22.32041677876001], Original value: 3514, Normalized value: 8.913434959169875
Coordinates: [114.16958289165, 22.34541677866001], Original value: 257, Normalized value: 6.062117

Coordinates: [114.20291622485, 22.287083445560008], Original value: 872, Normalized value: 7.392862719989633
Coordinates: [114.234072, 22.3646607], Original value: 0, Normalized value: 0.0
Coordinates: [114.16124955835, 22.26208344566001], Original value: 0, Normalized value: 0.0
Coordinates: [114.12791622515, 22.32041677876001], Original value: 954, Normalized value: 7.490870157591264
Coordinates: [114.21958289145, 22.278750112260006], Original value: 38, Normalized value: 3.999478154026489
Coordinates: [114.21958289145, 22.31208344546001], Original value: 4090, Normalized value: 9.079099010516812
Coordinates: [114.17791622495002, 22.27041677896001], Original value: 2371, Normalized value: 8.484066196399109
Coordinates: [114.24458289135003, 22.25375011236001], Original value: 0, Normalized value: 0.0
Coordinates: [114.24458289135003, 22.287083445560008], Original value: 0, Normalized value: 0.0
Coordinates: [114.30291622445004, 22.27041677896001], Original value: 2319, Normalized valu

Coordinates: [114.2454502477145, 22.24532179272471], Original value: 138, Normalized value: 5.386921963772238
Coordinates: [114.2277734841905, 22.35528189321145], Original value: 0, Normalized value: 0.0
Coordinates: [114.2934916096923, 22.2948900632964], Original value: 0, Normalized value: 0.0
Coordinates: [114.14458289175002, 22.245416779060008], Original value: 0, Normalized value: 0.0
Coordinates: [114.2347684026115, 22.35183197969195], Original value: 0, Normalized value: 0.0
Coordinates: [114.21124955815004, 22.36208344526001], Original value: 0, Normalized value: 0.0
Coordinates: [114.13624955845, 22.278750112260006], Original value: 0, Normalized value: 0.0
Coordinates: [114.1951245034016, 22.35227334422259], Original value: 201, Normalized value: 5.794989341563085
Coordinates: [114.1784134, 22.2951202], Original value: 0, Normalized value: 0.0
Coordinates: [114.19458289155, 22.33708344536001], Original value: 2959, Normalized value: 8.725827539196985
Coordinates: [114.1612495

Coordinates: [114.147188, 22.315451], Original value: 0, Normalized value: 0.0
Coordinates: [114.1852778, 22.2961111], Original value: 0, Normalized value: 0.0
Coordinates: [114.16124955835, 22.287083445560008], Original value: 2872, Normalized value: 8.693259673028445
Coordinates: [114.18624955824998, 22.32041677876001], Original value: 4562, Normalized value: 9.198301719072617
Coordinates: [114.2852643430797, 22.31183838953766], Original value: 8, Normalized value: 2.398690822039279
Coordinates: [114.121098, 22.3454483], Original value: 1376, Normalized value: 7.890375831093303
Coordinates: [114.16124955835, 22.35375011196001], Original value: 0, Normalized value: 0.0
Coordinates: [114.14458289175002, 22.287083445560008], Original value: 2687, Normalized value: 8.620597405725368
Coordinates: [114.152183, 22.345052], Original value: 0, Normalized value: 0.0
Coordinates: [114.16124955835, 22.328750112060007], Original value: 4060, Normalized value: 9.071063955308214
Coordinates: [114.1

Coordinates: [114.1195221, 22.3211155], Original value: 0, Normalized value: 0.0
Coordinates: [114.1692290352784, 22.37077042327222], Original value: 0, Normalized value: 0.0
Coordinates: [114.11958289185004, 22.36208344526001], Original value: 3017, Normalized value: 8.747011908143422
Coordinates: [114.3110289, 22.3711435], Original value: 0, Normalized value: 0.0
Coordinates: [114.21124955815004, 22.32041677876001], Original value: 4088, Normalized value: 9.078565176159287
Coordinates: [114.15291622505003, 22.328750112060007], Original value: 1934, Normalized value: 8.261769307253982
Coordinates: [114.076275, 22.2840883], Original value: 0, Normalized value: 0.0
Coordinates: [114.21958289145, 22.328750112060007], Original value: 3, Normalized value: 1.5134054092433589
Coordinates: [114.21124955815004, 22.29541677886001], Original value: 0, Normalized value: 0.0
Coordinates: [114.1520983231929, 22.29608152000107], Original value: 0, Normalized value: 0.0
Coordinates: [114.25291622465,

...

Coordinates: [140.26124945395003, 35.53708339256001], Original value: 1792, Normalized value: 8.178563732112735
Coordinates: [139.30291612445, 35.44541672626001], Original value: 92, Normalized value: 4.948199158316133
Coordinates: [139.59458278994998, 35.60375005896001], Original value: 475, Normalized value: 6.730732658612078
Coordinates: [140.2355471845269, 35.74567653523169], Original value: 0, Normalized value: 0.0
Coordinates: [140.05291612144998, 35.88708339116001], Original value: 6, Normalized value: 2.124333062353973
Coordinates: [139.55291612345002, 35.81208339146001], Original value: 2594, Normalized value: 8.582157968067362
Coordinates: [139.5954923732544, 35.85441519147993], Original value: 0, Normalized value: 0.0
Coordinates: [139.5357944085827, 35.68743935549224], Original value: 1161, Normalized value: 7.705045346745724
Coordinates: [140.1958405637877, 35.4543300069553], Original value: 1578, Normalized value: 8.03981145750157
Coordinates: [139.79458278915, 35.7954167

Coordinates: [139.26958279125, 35.86208339126001], Original value: 774, Normalized value: 7.262872285973155
Coordinates: [140.245494446196, 35.77911198643825], Original value: 253, Normalized value: 6.045059308850291
Coordinates: [139.96124945514998, 35.70375005856001], Original value: 5147, Normalized value: 9.329990236566085
Coordinates: [139.55291612345002, 35.52875005926001], Original value: 1766, Normalized value: 8.16261739677574
Coordinates: [139.58624945665002, 35.86208339126001], Original value: 0, Normalized value: 0.0
Coordinates: [139.78624945585, 35.80375005816001], Original value: 4391, Normalized value: 9.15660391963893
Coordinates: [139.551564761103, 35.44453186943073], Original value: 0, Normalized value: 0.0
Coordinates: [139.3006056174183, 35.71247416250168], Original value: 0, Normalized value: 0.0
Coordinates: [139.69458278955, 35.56208339246001], Original value: 4917, Normalized value: 9.280093019119219
Coordinates: [139.31124945775002, 35.53708339256001], Origina

Coordinates: [139.80291612245003, 35.77875005826001], Original value: 2434, Normalized value: 8.512682999507865
Coordinates: [139.2747774742327, 35.86125031953434], Original value: 174, Normalized value: 5.638351601030634
Coordinates: [140.2520559310348, 35.64526087322143], Original value: 0, Normalized value: 0.0
Coordinates: [139.32791612435, 35.81208339146001], Original value: 4092, Normalized value: 9.079632583958219
Coordinates: [139.96124945514998, 35.853750057960006], Original value: 4219, Normalized value: 9.112991317328671
Coordinates: [139.78624945585, 35.72041672516001], Original value: 4609, Normalized value: 9.20948888516033
Coordinates: [139.35291612425, 35.71208339186001], Original value: 4258, Normalized value: 9.123034072037928
Coordinates: [139.72791612275, 35.80375005816001], Original value: 4874, Normalized value: 9.270505962041481
Coordinates: [140.2477291748352, 35.81368122517575], Original value: 581, Normalized value: 6.950220013591673
Coordinates: [140.26958278

Coordinates: [140.24458278735, 35.88708339116001], Original value: 0, Normalized value: 0.0
Coordinates: [139.44458279055, 35.48708339276001], Original value: 3287, Normalized value: 8.840553599552361
Coordinates: [139.45291612385, 35.74541672506001], Original value: 5943, Normalized value: 9.486947072120241
Coordinates: [140.1294285909828, 35.7864756469609], Original value: 2993, Normalized value: 8.73829576412287
Coordinates: [139.36958279085002, 35.67875005866001], Original value: 3099, Normalized value: 8.776277695216514
Coordinates: [140.03624945485, 35.82875005806001], Original value: 677, Normalized value: 7.11689598250477
Coordinates: [139.49458279035002, 35.81208339146001], Original value: 2940, Normalized value: 8.71879747472414
Coordinates: [139.268193421956, 35.87710860965595], Original value: 0, Normalized value: 0.0
Coordinates: [139.50291612365004, 35.57041672576001], Original value: 1944, Normalized value: 8.267396594143715
Coordinates: [139.42791612395, 35.728750058460

Coordinates: [139.64458278975002, 35.74541672506001], Original value: 4112, Normalized value: 9.08495402388507
Coordinates: [140.2930625555529, 35.7879441332495], Original value: 951, Normalized value: 7.4874353632337565
Coordinates: [139.5026515346511, 35.82964630203206], Original value: 5395, Normalized value: 9.38135392577154
Coordinates: [139.60291612325, 35.57041672576001], Original value: 2986, Normalized value: 8.735740391387578
Coordinates: [139.48624945705, 35.43708339296001], Original value: 920, Normalized value: 7.45129492635209
Coordinates: [140.02791612155, 35.65375005876001], Original value: 3045, Normalized value: 8.757093561158856
Coordinates: [139.35291612425, 35.50375005936001], Original value: 4612, Normalized value: 9.210199082240162
Coordinates: [139.3634836444438, 35.74525063594564], Original value: 751, Normalized value: 7.2299832256126155
Coordinates: [139.47791612375, 35.81208339146001], Original value: 3478, Normalized value: 8.902196402776521
Coordinates: [1